Fine-tuning best T5 Transformer 🤖
-----------------------------------

In this notebook, we will continue the fine-tuning of T5 transformer on the new extracted sentences from the bool **Grammaire de Wolof Moderne** without considering the definitions. We obtained, after a hyperparameter tuning with `wandb`, a best bleu score of **4.281** for the french to wolof translation model. We provide, bellow, the main evaluation figures, obtained from the hyperparameter search step. It is important to notice that we will evaluate the training on the validation dataset.

- Parallel coordinates from panel:

- Parameter importance char: 
[t5_v3_importance](https://wandb.ai/oumar-kane-team/small-t5-cross-fw-translation-bayes-hpsearch-v3/reports/undefined-23-05-16-10-36-17---Vmlldzo0Mzc4NDY0?accessToken=eyaiyrid0qz1zg2jkq3fc65biw53084dpfitbi0dgonq6mweupw6kgjml9d2nv1w)

We can see in the above chart that the batch is the most important parameter with a negative correlation with the BLEU score (meaning that a lower batch size is better). Next, we the probability of modifying a character in the french corpus is also important and a high probability provide a better BLEU score.  

In [1]:
# let us import all necessary libraries
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, T5TokenizerFast, set_seed, AdamW, get_linear_schedule_with_warmup, T5ForConditionalGeneration,\
    get_cosine_schedule_with_warmup, Adafactor
from wolof_translate.utils.sent_transformers import TransformerSequences
from torch.nn import TransformerEncoderLayer, TransformerDecoderLayer
from torch.utils.data import Dataset, DataLoader, random_split
from wolof_translate.data.dataset_v2 import SentenceDataset
from wolof_translate.utils.sent_corrections import *
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import _LRScheduler
# from custom_rnn.utils.kwargs import Kwargs
from torch.nn.utils.rnn import pad_sequence
from plotly.subplots import make_subplots
from nlpaug.augmenter import char as nac
from torch.utils.data import DataLoader
# from datasets  import load_metric # make pip install evaluate instead
# and pip install sacrebleu for instance
from torch.nn import functional as F
import plotly.graph_objects as go
from tokenizers import Tokenizer
import matplotlib.pyplot as plt
from tqdm import tqdm, trange
from functools import partial
from torch.nn import utils
from copy import deepcopy
from torch import optim
from typing import *
from torch import nn
import pandas as pd
import numpy as np
import itertools
import evaluate
import random
import string
import shutil
import wandb
import torch
import json
import copy
import os

os.environ["WANDB_DISABLED"] = "true"

c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## French to wolof

### Configure dataset 🔠

In [2]:
# recuperate the tokenizer from a json file
tokenizer = T5TokenizerFast(tokenizer_file=f"wolof-translate/wolof_translate/tokenizers/t5_tokenizers/tokenizer_v3.json")


In [3]:
def recuperate_datasets(fr_char_p: float, fr_word_p: float):

  # Create augmentation to add on French sentences
  fr_augmentation = TransformerSequences(nac.KeyboardAug(aug_char_p=fr_char_p, aug_word_p=fr_word_p),
                                        remove_mark_space, delete_guillemet_space)

  # Recuperate the train dataset
  train_dataset_aug = SentenceDataset(f"data/extractions/new_data/train_set.csv",
                                        tokenizer,
                                        truncation = True,
                                        cp1_transformer = fr_augmentation)

  # Recuperate the valid dataset
  valid_dataset = SentenceDataset(f"data/extractions/new_data/valid_set.csv",
                                        tokenizer,
                                        truncation = True)
  
  # Return the datasets
  return train_dataset_aug, valid_dataset

### Configure the model and the evaluation function ⚙️

Let us evaluate the predictions with the `bleu` metric.

In [4]:
%%writefile wolof-translate/wolof_translate/utils/evaluation.py
from tokenizers import Tokenizer
from typing import *
import numpy as np
import evaluate

class TranslationEvaluation:
    
    def __init__(self, 
                 tokenizer: Tokenizer,
                 decoder: Union[Callable, None] = None,
                 metric = evaluate.load('sacrebleu'),
                 ):
        
        self.tokenizer = tokenizer
        
        self.decoder = decoder
        
        self.metric = metric
    
    def postprocess_text(self, preds, labels):
        
        preds = [pred.strip() for pred in preds]
        
        labels = [[label.strip()] for label in labels]
        
        return preds, labels

    def compute_metrics(self, eval_preds):

        preds, labels = eval_preds

        if isinstance(preds, tuple):
        
            preds = preds[0]
        
        decoded_preds = self.tokenizer.batch_decode(preds, skip_special_tokens=True)

        labels = np.where(labels != -100, labels, self.tokenizer.pad_token_id)
        
        decoded_labels = self.tokenizer.batch_decode(labels, skip_special_tokens=True)

        decoded_preds, decoded_labels = self.postprocess_text(decoded_preds, decoded_labels)

        result = self.metric.compute(predictions=decoded_preds, references=decoded_labels)
        
        result = {"bleu": result["score"]}

        prediction_lens = [np.count_nonzero(pred != self.tokenizer.pad_token_id) for pred in preds]
        
        result["gen_len"] = np.mean(prediction_lens)
        
        result = {k: round(v, 4) for k, v in result.items()}
        
        return result

Overwriting wolof-translate/wolof_translate/utils/evaluation.py


Let us initialize the evaluation object.

In [5]:
%run wolof-translate/wolof_translate/utils/evaluation.py
evaluation = TranslationEvaluation(tokenizer)


### Searching for the best parameters 🕖

In [6]:
from wolof_translate.models.transformers.optimization import TransformerScheduler
from wolof_translate.trainers.transformer_trainer import ModelRunner
from wolof_translate.utils.evaluation import TranslationEvaluation
from wolof_translate.models.transformers.main import Transformer
from wolof_translate.utils.split_with_valid import split_data


-------------

### ---

In [7]:
# let us initialize the hyperparameter configuration
config = {
    'random_state': 0,
    'fr_char_p': 0.5682074095811468,
    'fr_word_p': 0.026223984144846748,
    'learning_rate': 0.008879286602093731,
    'weight_decay': 0.5215535876633939,
    'batch_size': 32,
    'warmup_ratio': 0.0,
    'max_epoch': 956,
    'bleu': 4.881,
    'model_dir': 'data/checkpoints/fw_t5_small_custom_train_v3_checkpoints/',
    'new_model_dir': 'data/checkpoints/t5_small_custom_train_results_fw_v3/'
}

# Initialize the model name
model_name = 't5-small'

# import the model with its pre-trained weights
model = T5ForConditionalGeneration.from_pretrained(model_name)

# resize the token embeddings
model.resize_token_embeddings(len(tokenizer))

# let us initialize the evaluation class
evaluation = TranslationEvaluation(tokenizer)

# let us initialize the trainer
trainer = ModelRunner(model, seed = 0, version = 1, evaluation = evaluation)

# split the data
split_data(config['random_state'])

# recuperate train and test set
train_dataset, test_dataset = recuperate_datasets(config['fr_char_p'], 
                                                    config['fr_word_p'])

# let us calculate the appropriate warmup steps (let us take a max epoch of 100)
length = len(train_dataset)

n_steps = length // config['batch_size']

num_steps = config['max_epoch'] * n_steps

warmup_steps = (config['max_epoch'] * n_steps) * config['warmup_ratio']

# Initialize the scheduler parameters
scheduler_args = {'num_warmup_steps': warmup_steps, 'num_training_steps': num_steps}

# Initialize the optimizer parameters
optimizer_args = {
    'lr': config['learning_rate'],
    'weight_decay': config['weight_decay'],
    'betas': (0.9, 0.98),
}

# Initialize the loaders parameters
train_loader_args = {'batch_size': config['batch_size']}

# Add the datasets and hyperparameters to trainer
trainer.compile(train_dataset, test_dataset, tokenizer, train_loader_args,
                optimizer_kwargs = optimizer_args,
                lr_scheduler=get_linear_schedule_with_warmup,
                lr_scheduler_kwargs=scheduler_args, 
                predict_with_generate = True,
                hugging_face = True,
                logging_dir="data/logs/t5_small_custom_train_fw_v3"
                )

# We will from checkpoints so let us the model
# trainer.load(config['model_dir'], load_best=True) # Only for the first loading
trainer.load(config['new_model_dir'])

        

In [12]:
trainer.train(epochs = config['max_epoch'] - trainer.current_epoch, auto_save=True, metric_for_best_model='bleu', metric_objective='maximize', log_step=1,
              saving_directory = config['new_model_dir'])

  0%|          | 0/952 [00:00<?, ?it/s]c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\torch\optim\lr_scheduler.py:257: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


For epoch 5: {Learning rate: [0.008841205979637053]}


Test batch number 0:   0%|          | 0/10 [00:00<?, ?batches/s]c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\transformers\generation\utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Test batch number 9: 100%|██████████| 10/10 [00:03<00:00,  2.78batches/s]



Metrics: {'train_loss': 0.35802058911905055, 'test_loss': 0.3686817422509193, 'bleu': 4.1962, 'gen_len': 8.7192}




  0%|          | 1/952 [00:17<4:40:17, 17.68s/it]

...


Metrics: {'train_loss': 0.357867263439225, 'test_loss': 0.5150900736451149, 'bleu': 2.2111, 'gen_len': 6.2397}




  8%|▊         | 75/952 [21:16<8:31:38, 35.00s/it]

For epoch 80: {Learning rate: [0.008127194308574337]}


Test batch number 1:  10%|█         | 1/10 [00:02<00:21,  2.35s/batches]

: 

In [8]:
trainer.train(epochs = config['max_epoch'] - trainer.current_epoch, auto_save=True, metric_for_best_model='bleu', metric_objective='maximize', log_step=1,
              saving_directory = config['new_model_dir'])

  0%|          | 0/877 [00:00<?, ?it/s]c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\torch\optim\lr_scheduler.py:257: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


For epoch 80: {Learning rate: [0.008127194308574337]}


Test batch number 0:   0%|          | 0/10 [00:00<?, ?batches/s]c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\transformers\generation\utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Test batch number 9: 100%|██████████| 10/10 [00:03<00:00,  3.25batches/s]



Metrics: {'train_loss': 0.35787499823221347, 'test_loss': 0.49289117753505707, 'bleu': 2.3578, 'gen_len': 6.3562}




  0%|          | 1/877 [00:15<3:44:01, 15.34s/it]

...

Test batch number 9: 100%|██████████| 10/10 [00:03<00:00,  2.52batches/s]



Metrics: {'train_loss': 0.2347262730685676, 'test_loss': 0.46524187326431277, 'bleu': 7.6756, 'gen_len': 6.6712}




 47%|████▋     | 412/877 [1:37:30<2:01:18, 15.65s/it]

For epoch 492: {Learning rate: [0.00420489019553649]}


Train batch number 14:  34%|███▍      | 14/41 [00:03<00:06,  4.50batches/s]

### ---

In [8]:
trainer.train(epochs = config['max_epoch'] - trainer.current_epoch, auto_save=True, metric_for_best_model='bleu', metric_objective='maximize', log_step=1,
              saving_directory = config['new_model_dir'])

  0%|          | 0/465 [00:00<?, ?it/s]c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\torch\optim\lr_scheduler.py:257: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


For epoch 492: {Learning rate: [0.00420489019553649]}


Test batch number 0:   0%|          | 0/10 [00:00<?, ?batches/s]c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\transformers\generation\utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Test batch number 9: 100%|██████████| 10/10 [00:04<00:00,  2.40batches/s]



Metrics: {'train_loss': 0.23064579142303002, 'test_loss': 0.47452086210250854, 'bleu': 9.0274, 'gen_len': 7.1233}




  0%|          | 1/465 [00:15<1:57:45, 15.23s/it]

...

100%|█████████▉| 464/465 [3:01:20<00:15, 15.33s/it]

For epoch 956: {Learning rate: [0.0]}


Test batch number 9: 100%|██████████| 10/10 [00:04<00:00,  2.14batches/s]



Metrics: {'train_loss': 0.03899736888706684, 'test_loss': 0.4955804590135813, 'bleu': 17.2568, 'gen_len': 7.1096}




100%|██████████| 465/465 [3:01:35<00:00, 23.43s/it]


--------------

### ---

In [7]:
# let us initialize the hyperparameter configuration
config = {
    'random_state': 0,
    'fr_char_p': 0.2725135383958951,
    'fr_word_p': 0.8803557671452257,
    'learning_rate': 0.0007899243330013148,
    'weight_decay': 0.19140839339569107,
    'batch_size': 8,
    'warmup_ratio': 0.0,
    'max_epoch': 927,
    'bleu': 1.9136,
    'model_dir': 'data/checkpoints/fw_t5_small_custom_train_v3_2_checkpoints/',
    'new_model_dir': 'data/checkpoints/t5_small_custom_train_results_fw_v3_2/'
}

# Initialize the model name
model_name = 't5-small'

# import the model with its pre-trained weights
model = T5ForConditionalGeneration.from_pretrained(model_name)

# resize the token embeddings
model.resize_token_embeddings(len(tokenizer))

# let us initialize the evaluation class
evaluation = TranslationEvaluation(tokenizer)

# let us initialize the trainer
trainer = ModelRunner(model, seed = 0, version = 1, evaluation = evaluation, optimizer=Adafactor)

# split the data
split_data(config['random_state'])

# recuperate train and test set
train_dataset, test_dataset = recuperate_datasets(config['fr_char_p'], 
                                                    config['fr_word_p'])

# let us calculate the appropriate warmup steps (let us take a max epoch of 100)
length = len(train_dataset)

n_steps = length // config['batch_size']

num_steps = config['max_epoch'] * n_steps

warmup_steps = (config['max_epoch'] * n_steps) * config['warmup_ratio']

# Initialize the scheduler parameters
scheduler_args = {'num_warmup_steps': warmup_steps, 'num_training_steps': num_steps}

# Initialize the optimizer parameters
optimizer_args = {
    'lr': config['learning_rate'],
    'weight_decay': config['weight_decay'],
    # 'betas': (0.9, 0.98),
    'relative_step': False
}

# Initialize the loaders parameters
train_loader_args = {'batch_size': config['batch_size']}

# Add the datasets and hyperparameters to trainer
trainer.compile(train_dataset, test_dataset, tokenizer, train_loader_args,
                optimizer_kwargs = optimizer_args,
                lr_scheduler=get_cosine_schedule_with_warmup,
                lr_scheduler_kwargs=scheduler_args, 
                predict_with_generate = True,
                hugging_face = True,
                logging_dir="data/logs/t5_small_custom_train_fw_v3_2"
                )

# We will from checkpoints so let us the model
trainer.load(config['model_dir'], load_best=True) # Only for the first loading
# trainer.load(config['new_model_dir'])

        

### ---

In [8]:
trainer.train(epochs = config['max_epoch'] - trainer.current_epoch, auto_save=True, metric_for_best_model='bleu', metric_objective='maximize', log_step=1,
              saving_directory = config['new_model_dir'])

  0%|          | 0/467 [00:00<?, ?it/s]c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\torch\optim\lr_scheduler.py:257: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


For epoch 4: {Learning rate: [0.0006099460743927353]}


Test batch number 0:   0%|          | 0/10 [00:00<?, ?batches/s]c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\transformers\generation\utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Test batch number 9: 100%|██████████| 10/10 [00:03<00:00,  2.55batches/s]



Metrics: {'train_loss': 0.6670737142969922, 'test_loss': 0.6173947781324387, 'bleu': 0.7582, 'gen_len': 6.9863}




  0%|          | 1/467 [00:35<4:33:30, 35.21s/it]

...

For epoch 30: {Learning rate: [0.005896145385796442]}


Test batch number 9: 100%|██████████| 10/10 [00:04<00:00,  2.27batches/s]



Metrics: {'train_loss': 0.32658602651662944, 'test_loss': 0.48370583057403566, 'bleu': 1.9209, 'gen_len': 5.2671}




  6%|▌         | 27/467 [14:56<4:00:57, 32.86s/it]

For epoch 31: {Learning rate: [0.006099460743927354]}


### ---

In [8]:
trainer.train(epochs = config['max_epoch'] - trainer.current_epoch, auto_save=True, metric_for_best_model='bleu', metric_objective='maximize', log_step=1,
              saving_directory = config['new_model_dir'])

  0%|          | 0/924 [00:00<?, ?it/s]c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\torch\optim\lr_scheduler.py:257: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


For epoch 4: {Learning rate: [0.0007899036688778694]}


Test batch number 1:   0%|          | 0/10 [00:00<?, ?batches/s]c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\transformers\generation\utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Test batch number 10: 100%|██████████| 10/10 [00:03<00:00,  2.65batches/s]



Metrics: {'train_loss': 0.5944981680410665, 'test_loss': 0.5867837905883789, 'bleu': 0.9198, 'gen_len': 6.3082}




  0%|          | 1/924 [00:45<11:33:45, 45.10s/it]

...

Test batch number 10: 100%|██████████| 10/10 [00:03<00:00,  2.68batches/s]



Metrics: {'train_loss': 0.021213641438474196, 'test_loss': 0.54854137301445, 'bleu': 9.8897, 'gen_len': 7.3082}




  9%|▉         | 85/924 [59:13<10:38:01, 45.63s/it]

For epoch 89: {Learning rate: [0.0007722768555671693]}


Train batch number 29:  17%|█▋        | 28/164 [00:06<00:32,  4.21batches/s]

In [9]:
# let us get the best model
# model = T5ForConditionalGeneration.from_pretrained('data/checkpoints/t5_results_fw_v3/...')

# let us get the test set
test_dataset = SentenceDataset(f"data/extractions/new_data/test_set.csv",
                                        tokenizer,
                                        truncation = True)

### Predictions and Evaluation

Let us generate texts and store into a DataFrame.

In [11]:
df_ft_to_wf.tail(10)

,original_text,original_label,predicted_label
152,"Homme, lion, boeuf... allaient de concert.","Nit, gayndé, nag... àndoon nañu fi.","Nit, gayndé, nag, àndoon nañu fi."
153,C'est toi qui eusses été élu,Yaa doonkoon falu,Yaa doonkoon wax
154,L'homme ne cultivera pas,Góor gi du bày,Góor gi bëggul
155,S'agiter simplement ne suffit à rien résoudre.,Di tel-teli doŋŋ taxul sotal dara.,Nit ñenn ñi yegseeguñu.
156,C'était son hôte habituellement.,Moo doon ganam.,Man xar mépp.
157,Je parle de ceux-là!,Yenn xar yooyuu laa wax!,Yaw moomu laa wax
158,Tu reconnais cet enfant-ci?,Xammee ŋga bee xale?,Xammee ŋga waa jooju?
159,"Alors l'homme entra, les enfants le virent, il...","Noona góor gi dugg, xale yi gis ka, mu toog, ñ...","Noona Góor gaa ŋgi, mu ñëw."
160,C'est leur ami!,Suñu xarit la!,Su demee
161,Il était Lebou de Yoff.,Mu doon Lebu Yoff.,Dafa doon nitu dëgg.


In [12]:
# let us display 100 samples
pd.options.display.max_rows = 100
df_ft_to_wf.sample(100)

,original_text,original_label,predicted_label
105,Qui est-ce?,Ñan la?,Ku mu?
80,Tu as dit cela.,La ŋga wax la.,Li ŋga wax loolu.
52,A Moussa!,Musaa!,Musaa
132,Je connais l'enfant.,Xam naa xale bi.,Xam naa xale bi.
59,L'homme qui eût travaillé,Waa ji liggéeykoon,Góor gi waxkoon na
54,Le voilà qui part!,Mi ŋgiiy!,Ma ŋgee doon dem
115,Que tu partes ou que tu ne partes pas il viendra.,Dana ñëw soo demul ag soo demee itam.,"Soo demee ag soo demul itam, dana ñëw."
114,C'est l'homme qui a soutenu qu'il est sain d'e...,"Góor gee ni nit la, soo demee!",Góor gee ni soo demee nit la
46,J'ai vu mes amis!,Gis naa sana xarit yi!,Gis naa sama xarit yeneen yooyuu
147,Appelle l'homme qui ne part pas,Wool góor gi dul dem,Wool góor gi dul dem


## Colab download and remove step

In [ ]:
import shutil

# shutil.rmtree('/content/drive/MyDrive/Memoire/subject2/training2/results2')
shutil.rmtree('wandb')
# shutil.make_archive('wandb', 'zip', 'wanbd')